In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler


df = pd.read_excel("gym recommendation.xlsx")


df.drop(columns=["ID", "BMI", "Recommendation"], inplace=True)


df["Sex"] = df["Sex"].map({"Male": 1, "Female": 0})
df["Hypertension"] = df["Hypertension"].map({"Yes": 1, "No": 0})
df["Diabetes"] = df["Diabetes"].map({"Yes": 1, "No": 0})


le_level = LabelEncoder()
le_goal = LabelEncoder()
le_type = LabelEncoder()

df["Level"] = le_level.fit_transform(df["Level"])
df["Fitness Goal"] = le_goal.fit_transform(df["Fitness Goal"])
df["Fitness Type"] = le_type.fit_transform(df["Fitness Type"])


from sklearn.preprocessing import MinMaxScaler


minmax_scaler = MinMaxScaler()


df[["Age", "Height", "Weight"]] = minmax_scaler.fit_transform(df[["Age", "Height", "Weight"]])

print("Sex Mapping: {'Male': 1, 'Female': 0}")
print("Hypertension Mapping: {'Yes': 1, 'No': 0}")
print("Diabetes Mapping: {'Yes': 1, 'No': 0}")

print("\nLevel Encoding Mapping:")
for i, label in enumerate(le_level.classes_):
    print(f"{i}: {label}")

print("\nFitness Goal Encoding Mapping:")
for i, label in enumerate(le_goal.classes_):
    print(f"{i}: {label}")

print("\nFitness Type Encoding Mapping:")
for i, label in enumerate(le_type.classes_):
    print(f"{i}: {label}")



Sex Mapping: {'Male': 1, 'Female': 0}
Hypertension Mapping: {'Yes': 1, 'No': 0}
Diabetes Mapping: {'Yes': 1, 'No': 0}

Level Encoding Mapping:
0: Normal
1: Obuse
2: Overweight
3: Underweight

Fitness Goal Encoding Mapping:
0: Weight Gain
1: Weight Loss

Fitness Type Encoding Mapping:
0: Cardio Fitness
1: Muscular Fitness


Exercise_Column



In [7]:

df["Exercises"] = df["Exercises"].str.lower()
import re

df["Exercises"] = (
    df["Exercises"]
    .str.replace(r"\band\b", ",", flags=re.IGNORECASE, regex=True)
    .str.replace(r"\bor\b", ",", flags=re.IGNORECASE, regex=True)
    .str.replace(r",\s*,", ",", regex=True)
    .str.replace(r"\s*,\s*", ",", regex=True)
    .str.replace(r"\.$", "", regex=True)
    .str.strip()
)

unique_exercises = (
    df["Exercises"]
    .dropna()
    .str.split(",")
    .explode()
    .str.strip()
    .unique()
)


unique_exercises = sorted(unique_exercises)


print("Unique exercise items in the 'Exercises' column:\n")
for item in unique_exercises:
    print("-", item)


all_exercises = (
    df["Exercises"]
    .dropna()
    .str.split(",")
    .explode()
    .str.strip()
)


unique_exercise_items = sorted(all_exercises.unique())


exercise_mapping = {item: idx for idx, item in enumerate(unique_exercise_items)}


print("Exercise to Numeric Mapping:\n")
for name, code in exercise_mapping.items():
    print(f"{code}: {name}")

def encode_exercise_list(text):
    if pd.isna(text):
        return []
    items = [i.strip() for i in text.split(",")]
    return [exercise_mapping[i] for i in items if i in exercise_mapping]

df["Exercises_encoded"] = df["Exercises"].apply(encode_exercise_list)



unique_exercise_lists = df["Exercises_encoded"].dropna().apply(lambda x: tuple(x) if isinstance(x, list) else ()).unique()


unique_exercise_lists = sorted(unique_exercise_lists)

print("Unique encoded exercise lists:")
for eq_list in unique_exercise_lists:
    print("-", list(eq_list))



df["Exercises_encoded_tuple"] = df["Exercises_encoded"].apply(
    lambda x: tuple(x) if isinstance(x, list) else ()
)


unique_exercise_list_mapping = {
    tpl: idx for idx, tpl in enumerate(sorted(df["Exercises_encoded_tuple"].unique()))
}


print("Mapping from Exercise List to Unique ID:\n")
for tpl, idx in unique_exercise_list_mapping.items():
    print(f"{idx}: {list(tpl)}")


df["Exercises"] = df["Exercises_encoded_tuple"].apply(
    lambda x: unique_exercise_list_mapping.get(x, -1)
)


df.drop(columns=["Exercises_encoded", "Exercises_encoded_tuple"], inplace=True)



Unique exercise items in the 'Exercises' column:

- bench presses
- brisk walking
- cycling
- dancing
- deadlifts
- overhead presses
- running
- squats
- swimming
- walking
- yoga
Exercise to Numeric Mapping:

0: bench presses
1: brisk walking
2: cycling
3: dancing
4: deadlifts
5: overhead presses
6: running
7: squats
8: swimming
9: walking
10: yoga
Unique encoded exercise lists:
- [1, 2, 8, 3]
- [1, 2, 8, 6, 3]
- [7, 4, 0, 5]
- [7, 10, 4, 0, 5]
- [9, 10, 8]
Mapping from Exercise List to Unique ID:

0: [1, 2, 8, 3]
1: [1, 2, 8, 6, 3]
2: [7, 4, 0, 5]
3: [7, 10, 4, 0, 5]
4: [9, 10, 8]


Equipment_Col


In [8]:

df["Equipment"] = df["Equipment"].str.lower()

import re
df["Equipment"] = (
    df["Equipment"]
    .str.replace(r"\s+and\s+a\s+blood glucose monitor", ",blood glucose monitor", flags=re.IGNORECASE, regex=True)
    .str.replace(r"\s+and\s+blood glucose monitor", ",blood glucose monitor", flags=re.IGNORECASE, regex=True)
    .str.replace(r"\s+and\s+", ",", regex=True)
    .str.replace(r",\s*,", ",", regex=True)
    .str.replace(r"\.$", "", regex=True)
    .str.strip()
)

all_items = (
    df["Equipment"]
    .dropna()
    .str.lower()
    .str.split(",")
    .explode()
    .str.strip()
)


unique_items = sorted(all_items.unique())


print("Unique equipment items in the 'Equipment' column:")
for item in unique_items:
    print("-", item)


equipment_mapping = {item: idx for idx, item in enumerate(unique_items)}


print("Equipment to Numeric Mapping:")
for item, code in equipment_mapping.items():
    print(f"{code}: {item}")


def encode_equipment(equipment_string):
    if pd.isna(equipment_string):
        return []
    items = [i.strip().lower() for i in equipment_string.split(",")]
    return [equipment_mapping[i] for i in items if i in equipment_mapping]

df["Equipment_encoded"] = df["Equipment"].apply(encode_equipment)





unique_encoded_lists = df["Equipment_encoded"].dropna().apply(tuple).unique()


unique_encoded_lists = sorted(unique_encoded_lists)


print("Unique encoded equipment lists:")
for eq_list in unique_encoded_lists:
    print("-", list(eq_list))



df["Equipment_encoded_tuple"] = df["Equipment_encoded"].apply(lambda x: tuple(x) if isinstance(x, list) else ())


unique_lists = df["Equipment_encoded_tuple"].dropna().unique()
equipment_list_mapping = {tpl: idx for idx, tpl in enumerate(sorted(unique_lists))}


df["Equipment"] = df["Equipment_encoded_tuple"].apply(lambda x: equipment_list_mapping.get(x, -1))


print("Mapping from Equipment list to ID:\n")
for lst, idx in equipment_list_mapping.items():
    print(f"{idx}: {list(lst)}")


df.drop(columns=["Equipment_encoded", "Equipment_encoded_tuple"], inplace=True)

Unique equipment items in the 'Equipment' column:
- barbells
- blood glucose monitor
- dumbbells
- ellipticals
- equipment required
- indoor rowers
- kettlebell
- light athletic shoes
- light dumbbells
- resistance bands
- rowing machine
- treadmill
- treadmills
- yoga mat
Equipment to Numeric Mapping:
0: barbells
1: blood glucose monitor
2: dumbbells
3: ellipticals
4: equipment required
5: indoor rowers
6: kettlebell
7: light athletic shoes
8: light dumbbells
9: resistance bands
10: rowing machine
11: treadmill
12: treadmills
13: yoga mat
Unique encoded equipment lists:
- [2, 0]
- [2, 0, 1]
- [3, 5, 12, 10]
- [4]
- [6, 2, 13]
- [6, 2, 13, 11]
- [7, 9, 8]
- [7, 9, 8, 1]
Mapping from Equipment list to ID:

0: [2, 0]
1: [2, 0, 1]
2: [3, 5, 12, 10]
3: [4]
4: [6, 2, 13]
5: [6, 2, 13, 11]
6: [7, 9, 8]
7: [7, 9, 8, 1]


Drop diet

In [9]:
df.drop(columns=["Diet"], inplace=True)


In [10]:

df.to_excel("testsss.xlsx", index=False)

print("Data cleaning complete.")

Data cleaning complete.


Define Inputs and Target(exercise
)

In [11]:

X = df.drop(columns=["Exercises", "Equipment", "Level"])


y = df[["Exercises", "Equipment"]]


Split the Dataset

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Initialize Multi-Output Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier


multi_model = MultiOutputClassifier(RandomForestClassifier(random_state=42))


multi_model.fit(X_train, y_train)


MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

Choose a Model (e.g., Random Forest)

Make Predictions

In [14]:

y_pred = multi_model.predict(X_test)


Evaluate the model

In [15]:
from sklearn.metrics import accuracy_score, classification_report


y_pred_exercise = y_pred[:, 0]
y_pred_equipment = y_pred[:, 1]


y_test_exercise = y_test["Exercises"]
y_test_equipment = y_test["Equipment"]


print("Exercises Prediction Report:\n")
print("Accuracy:", accuracy_score(y_test_exercise, y_pred_exercise))
print(classification_report(y_test_exercise, y_pred_exercise))


print("Equipment Prediction Report:\n")
print("Accuracy:", accuracy_score(y_test_equipment, y_pred_equipment))
print(classification_report(y_test_equipment, y_pred_equipment))


Exercises Prediction Report:

Accuracy: 0.9962302947224126
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       0.99      1.00      0.99       774
           2       1.00      0.99      0.99       716
           3       1.00      1.00      1.00       725
           4       1.00      0.99      1.00       370

    accuracy                           1.00      2918
   macro avg       1.00      1.00      1.00      2918
weighted avg       1.00      1.00      1.00      2918

Equipment Prediction Report:

Accuracy: 0.9605894448252228
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       394
           1       0.96      0.95      0.95       348
           2       0.97      0.97      0.97       780
           3       0.86      0.40      0.55        15
           4       0.96      0.96      0.96       365
           5       0.97      0.99      0.98       332
           6  

In [16]:



reverse_exercise_id_to_list = {v: k for k, v in unique_exercise_list_mapping.items()}
reverse_exercise_mapping = {i: name for name, i in exercise_mapping.items()}


reverse_equipment_id_to_list = {v: k for k, v in equipment_list_mapping.items()}
reverse_equipment_mapping = {i: name for name, i in equipment_mapping.items()}


y_pred = multi_model.predict(X_test)
y_pred_exercise = y_pred[:, 0]
y_pred_equipment = y_pred[:, 1]


exercise_names = []
for exercise_id in y_pred_exercise:
    exercise_code_list = reverse_exercise_id_to_list.get(exercise_id, [])
    readable_exercises = [reverse_exercise_mapping.get(code, "Unknown") for code in exercise_code_list]
    exercise_names.append(",".join(readable_exercises))


equipment_names = []
for equip_id in y_pred_equipment:
    equip_code_list = reverse_equipment_id_to_list.get(equip_id, [])
    readable_equipment = [reverse_equipment_mapping.get(code, "Unknown") for code in equip_code_list]
    equipment_names.append(",".join(readable_equipment))


print("\n Sample Predictions:")
for i in range(5):
    print(f"Predicted Exercise: {exercise_names[i]}")
    print(f"Predicted Equipment: {equipment_names[i]}")
    print("-" * 50)



 Sample Predictions:
Predicted Exercise: brisk walking,cycling,swimming,running,dancing
Predicted Equipment: ellipticals,indoor rowers,treadmills,rowing machine
--------------------------------------------------
Predicted Exercise: squats,deadlifts,bench presses,overhead presses
Predicted Equipment: dumbbells,barbells
--------------------------------------------------
Predicted Exercise: squats,yoga,deadlifts,bench presses,overhead presses
Predicted Equipment: light athletic shoes,resistance bands,light dumbbells,blood glucose monitor
--------------------------------------------------
Predicted Exercise: walking,yoga,swimming
Predicted Equipment: kettlebell,dumbbells,yoga mat
--------------------------------------------------
Predicted Exercise: squats,deadlifts,bench presses,overhead presses
Predicted Equipment: dumbbells,barbells
--------------------------------------------------


SAVE

In [17]:
import joblib
import pickle


joblib.dump(multi_model, "gym_recommender_model.pkl")


joblib.dump(minmax_scaler, "minmax_scaler.pkl")



joblib.dump(le_level, "level_encoder.pkl")
joblib.dump(le_goal, "goal_encoder.pkl")
joblib.dump(le_type, "type_encoder.pkl")


with open("exercise_mapping.pkl", "wb") as f:
    pickle.dump(exercise_mapping, f)

with open("unique_exercise_list_mapping.pkl", "wb") as f:
    pickle.dump(unique_exercise_list_mapping, f)

with open("equipment_mapping.pkl", "wb") as f:
    pickle.dump(equipment_mapping, f)

with open("equipment_list_mapping.pkl", "wb") as f:
    pickle.dump(equipment_list_mapping, f)


predict


In [18]:
import joblib
import pickle
import pandas as pd


model = joblib.load("gym_recommender_model.pkl")
scaler = joblib.load("minmax_scaler.pkl")

le_level = joblib.load("level_encoder.pkl")
le_goal = joblib.load("goal_encoder.pkl")
le_type = joblib.load("type_encoder.pkl")

with open("exercise_mapping.pkl", "rb") as f:
    exercise_mapping = pickle.load(f)

with open("unique_exercise_list_mapping.pkl", "rb") as f:
    unique_exercise_list_mapping = pickle.load(f)

with open("equipment_mapping.pkl", "rb") as f:
    equipment_mapping = pickle.load(f)

with open("equipment_list_mapping.pkl", "rb") as f:
    equipment_list_mapping = pickle.load(f)


reverse_exercise_id_to_list = {v: k for k, v in unique_exercise_list_mapping.items()}
reverse_exercise_mapping = {v: k for k, v in exercise_mapping.items()}

reverse_equipment_id_to_list = {v: k for k, v in equipment_list_mapping.items()}
reverse_equipment_mapping = {v: k for k, v in equipment_mapping.items()}


input_raw = {
    'Sex': 1,
    'Age': 30,
    'Height': 175,
    'Weight': 70,
    'Hypertension': 0,
    'Diabetes': 0,
    'Fitness Goal': 'Weight Loss',
    'Fitness Type': 'Cardio Fitness'
}


input_raw['Fitness Goal'] = le_goal.transform([input_raw['Fitness Goal']])[0]
input_raw['Fitness Type'] = le_type.transform([input_raw['Fitness Type']])[0]


X_test = pd.DataFrame([[
    input_raw['Sex'],
    input_raw['Age'],
    input_raw['Height'],
    input_raw['Weight'],
    input_raw['Hypertension'],
    input_raw['Diabetes'],
    input_raw['Fitness Goal'],
    input_raw['Fitness Type'],
]], columns=['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'Fitness Goal', 'Fitness Type'])


numerical_cols = ["Age", "Height", "Weight"]


X_test_scaled = X_test.copy()


X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])


y_pred = model.predict(X_test_scaled)


exercise_pred = y_pred[0][0]
equipment_pred = y_pred[0][1]


exercise_codes = reverse_exercise_id_to_list.get(exercise_pred, ())
exercise_names = [reverse_exercise_mapping.get(code, "Unknown") for code in exercise_codes]


equipment_codes = reverse_equipment_id_to_list.get(equipment_pred, ())
equipment_names = [reverse_equipment_mapping.get(code, "Unknown") for code in equipment_codes]

print("Recommended Exercises:", ", ".join(exercise_names))
print("Recommended Equipment:", ", ".join(equipment_names))


Recommended Exercises: brisk walking, cycling, swimming, running, dancing
Recommended Equipment: ellipticals, indoor rowers, treadmills, rowing machine


In [19]:
def predict_gym_plan(sex, age, height, weight, hypertension, diabetes, level, goal, ftype,
                     model, scaler, le_level, le_goal, le_type,
                     reverse_exercise_id_to_list, reverse_exercise_mapping,
                     reverse_equipment_id_to_list, reverse_equipment_mapping):

    level_encoded = le_level.transform([level])[0]
    goal_encoded = le_goal.transform([goal])[0]
    type_encoded = le_type.transform([ftype])[0]


    height_m = height / 100
    bmi = calculate_bmi(weight, height_m)

    input_df = pd.DataFrame([[
        sex,
        age,
        height,
        weight,
        hypertension,
        diabetes,
        goal_encoded,
        type_encoded
    ]], columns=['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'Fitness Goal', 'Fitness Type'])


    input_df_scaled = input_df.copy()
    input_df_scaled[['Age', 'Height', 'Weight']] = scaler.transform(input_df[['Age', 'Height', 'Weight']])


    prediction = model.predict(input_df_scaled)
    exercise_id, equipment_id = prediction[0]


    exercise_codes = reverse_exercise_id_to_list.get(exercise_id, [])
    exercise_names = [reverse_exercise_mapping.get(code, "Unknown") for code in exercise_codes]


    equipment_codes = reverse_equipment_id_to_list.get(equipment_id, [])
    equipment_names = [reverse_equipment_mapping.get(code, "Unknown") for code in equipment_codes]

    return {
        "bmi": round(bmi, 2),
        "exercise_names": exercise_names,
        "equipment_names": equipment_names
    }
